<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d3c757976a57b12742ea4a6dab3f4bff37e31987931969e862f755d3d35f9a96
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 16:02:47
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 30.40 K (0.21%)
Current PnL: -16.74 L (-11.26%)
CY Booked + Current PnL: -5.30 L (-3.56%)
-------------------
Total profit:  2.11 L
Total loss:  -18.85 L
-------------------
Total Booked + Current PnL: 21.68 L (17.82%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.27 L (56.89%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.61,65.0,H-MC,3.57,196100.0,31430.0,11903.0,1.07,19.09,6.07,26.31,89.0,2.64,1.43,28.59,X40N,NTT,AC
76,TTKPRESTIG,770.00,104.91,58.0,M-SC,3.97,88242.0,-12535.0,12627.0,-0.47,-12.44,14.31,0.09,245.0,-0.99,0.65,15.93,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,2.00,61.0,H-MC,2.51,213930.0,22188.0,15574.0,1.01,11.57,7.28,19.69,99.0,1.42,1.56,19.16,XY25,NTT,AC
49,NATIONALUM,247.44,-42.83,65.0,H-MC,6.84,113314.0,12810.0,17824.0,0.25,12.75,15.73,30.49,79.0,0.72,0.83,49.28,MH,ATH,METALS
56,RELIANCE,1533.00,-11.79,60.0,H-LC,5.19,220740.0,10374.0,18410.0,0.08,4.93,8.34,13.68,37.0,0.56,1.61,22.27,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.46,56.0,H-SC,17.06,201664.0,1710.0,95528.0,0.11,0.86,47.37,48.63,115.0,0.02,1.48,12.51,XR,NTT,MISC
34,ICICIPRULI,790.00,-20.54,46.0,H-MC,2.22,136730.0,942.0,41812.0,0.82,0.69,30.58,31.48,107.0,0.02,1.00,12.90,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.34,59.0,H-MC,1.88,248711.0,980.0,74066.0,0.05,0.40,29.78,30.30,92.0,0.01,1.82,13.74,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.81,52.0,H-MC,4.21,200295.0,-525.0,69302.0,-0.23,-0.26,34.60,34.25,80.0,-0.01,1.47,4.92,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-16.13,42.0,H-LC,6.33,302125.0,-13717.0,126379.0,0.53,-4.34,41.83,35.67,5.0,-0.11,2.21,8.91,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.58,44.0,H-LC,7.80,218126.0,-33642.0,86858.0,-0.60,-13.36,39.82,21.13,27.0,-0.39,1.60,17.33,X40,ATH,PAINTS
41,ITC,452.00,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.47,6.15,X40,NTT,FMCG
79,UNITDSPR,1644.00,-12.94,52.0,H-LC,7.80,228313.0,-5831.0,54453.0,-0.73,-2.49,23.85,20.77,86.0,-0.11,1.67,4.26,X40N,NTT,BREWERIES
43,JIOFIN,387.00,0.47,53.0,H-LC,13.23,272921.0,5556.0,59906.0,0.47,2.08,21.95,24.48,48.0,0.09,2.00,58.19,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.29,70.0,H-LC,1.27,248602.0,2451.0,76768.0,1.87,1.00,30.88,32.18,16.0,0.03,1.82,37.40,X200,ATH,IT
41,ITC,452.0,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.47,6.15,X40,NTT,FMCG
50,NESTLEIND,1377.0,-8.10,58.0,H-LC,4.23,282929.0,17503.0,39299.0,0.40,6.59,13.89,21.40,11.0,0.45,2.07,14.16,XY25,NTT,FMCG
56,RELIANCE,1533.0,-11.79,60.0,H-LC,5.19,220740.0,10374.0,18410.0,0.08,4.93,8.34,13.68,37.0,0.56,1.61,22.27,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.09,56.0,H-LC,5.81,216213.0,11713.0,29708.0,1.19,5.73,13.74,20.25,10.0,0.39,1.58,14.96,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6966.67,64.0,L-SC,5.26,77563.0,-16247.0,92866.0,4.68,-17.32,119.73,81.67,269.0,-0.17,0.57,50.20,XR,NTT,CERAMICS
63,SHALBY,327.00,1099.36,68.0,M-SC,3.64,163077.0,-18066.0,63535.0,3.28,-9.97,38.96,25.10,235.0,-0.28,1.19,30.69,XY24,NTT,HEALTHCARE
60,SAMMAANCAP,326.00,-172.73,65.0,M-SC,2.67,83514.0,-18696.0,112084.0,3.05,-18.29,134.21,91.37,208.0,-0.17,0.61,35.57,XY25,NTT,FINANCE
81,VALIANTORG,838.00,-328.95,57.0,H-SC,6.53,127347.0,-36258.0,157579.0,2.07,-22.16,123.74,74.15,139.0,-0.23,0.93,63.24,XR,NTT,CHEMICALS
15,BSOFT,836.99,-12.08,58.0,H-SC,15.01,99004.0,-41649.0,114429.0,1.90,-29.61,115.58,51.74,131.0,-0.36,0.72,12.90,XR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GREENPANEL,537.00,228.81,59.0,M-SC,2.51,146717.0,-36361.0,115334.0,-3.00,-19.86,78.61,43.14,230.0,-0.32,1.07,38.01,XY24,NTT,MISC
54,RAJOOENG,143.33,-37.02,36.0,H-SC,11.64,95490.0,-7010.0,47840.0,-2.56,-6.84,50.10,39.83,114.0,-0.15,0.70,0.00,AR,ATH,MISC
57,REPCOHOME,880.00,-59.76,52.0,H-SC,6.44,180473.0,-55270.0,244559.0,-1.81,-23.44,135.51,80.30,134.0,-0.23,1.32,22.17,XY24,NTT,FINANCE
20,COALINDIA,497.97,8.96,59.0,L-LC,2.37,145466.0,-4994.0,38781.0,-1.65,-3.32,26.66,22.46,167.0,-0.13,1.06,14.16,XY25,ATH,MINING
45,KPIGREEN,731.05,1.73,41.0,H-SC,10.00,121174.0,-4123.0,63047.0,-1.27,-3.29,52.03,47.03,141.0,-0.07,0.89,51.72,MH,ATH,POWER


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,104.91,58.0,M-SC,3.97,88242.0,-12535.0,12627.0,-0.47,-12.44,14.31,0.09,245.0,-0.99,0.65,15.93,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,94.87,53.0,H-SC,9.51,122877.0,-19602.0,39124.0,-0.52,-13.76,31.84,13.70,163.0,-0.50,0.90,45.03,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.87,65.0,H-SC,1.38,227925.0,-41742.0,78064.0,1.51,-15.48,34.25,13.47,138.0,-0.53,1.67,15.28,XY24,NTT,PAINTS
18,CERA,9475.0,-19.81,47.0,H-SC,2.10,145992.0,-29911.0,71930.0,-0.42,-17.00,49.27,23.89,149.0,-0.42,1.07,26.08,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.51,55.0,H-MC,7.65,108936.0,-22899.0,65122.0,0.89,-17.37,59.78,32.03,98.0,-0.35,0.80,22.02,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,60.0,H-LC,12.33,298610.0,-47346.0,122191.0,0.12,-13.69,40.92,21.64,1.0,-0.39,2.18,5.78,X40,ATH,IT
39,INFY,2275.00,-15.65,58.0,H-LC,8.42,328148.0,15260.0,156428.0,1.20,4.88,47.67,54.87,3.0,0.10,2.40,11.85,X40,BTT,IT
41,ITC,452.00,-37.50,51.0,H-LC,2.52,201248.0,1110.0,19783.0,0.55,0.55,9.83,10.44,4.0,0.06,1.47,6.15,X40,NTT,FMCG
82,VBL,671.64,-16.13,42.0,H-LC,6.33,302125.0,-13717.0,126379.0,0.53,-4.34,41.83,35.67,5.0,-0.11,2.21,8.91,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,10.34,260928.0,-691.0,119896.0,0.93,-0.26,45.95,45.57,7.0,-0.01,1.91,11.03,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,45.15,66.0,M-MC,6.97,229336.0,4374.0,163287.0,0.32,1.94,71.20,74.53,192.0,0.03,1.68,34.82,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.61,65.0,H-MC,3.57,196100.0,31430.0,11903.0,1.07,19.09,6.07,26.31,89.0,2.64,1.43,28.59,X40N,NTT,AC
83,VOLTAS,1530.00,2.00,61.0,H-MC,2.51,213930.0,22188.0,15574.0,1.01,11.57,7.28,19.69,99.0,1.42,1.56,19.16,XY25,NTT,AC
17,CAMS,5211.76,0.38,67.0,H-SC,3.81,114106.0,12102.0,31824.0,1.53,11.86,27.89,43.06,122.0,0.38,0.83,31.01,X40N,ATH,MISC
29,HAVELLS,2069.16,0.34,59.0,H-MC,1.88,248711.0,980.0,74066.0,0.05,0.40,29.78,30.30,92.0,0.01,1.82,13.74,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,65.0,H-MC,6.84,113314.0,12810.0,17824.0,0.25,12.75,15.73,30.49,79.0,0.72,0.83,49.28,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.79,51.0,M-SC,0.48,100155.0,8656.0,70098.0,-0.46,9.46,69.99,86.07,223.0,0.12,0.73,45.44,XR,NTT,DURABLES
36,INDIAMART,4850.92,-52.03,41.0,H-SC,11.65,133719.0,10383.0,123383.0,-1.09,8.42,92.27,108.45,119.0,0.08,0.98,32.65,AR,ATH,MISC
85,WIPRO,420.00,-9.26,60.0,M-LC,5.63,159297.0,8352.0,101106.0,1.09,5.53,63.47,72.51,53.0,0.08,1.17,11.89,XR,NTT,IT
11,BANKINDIA,190.00,-29.19,63.0,H-MC,6.11,183037.0,3229.0,109182.0,-0.53,1.80,59.65,62.52,88.0,0.03,1.34,36.50,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,RAJESHEXPO,518.00,1891.16,67.0,L-SC,3.06,54314.0,-82863.0,82954.0,0.06,-60.41,152.73,0.07,267.0,-1.00,0.40,35.08,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,6966.67,64.0,L-SC,5.26,77563.0,-16247.0,92866.0,4.68,-17.32,119.73,81.67,269.0,-0.17,0.57,50.20,XR,NTT,CERAMICS
60,SAMMAANCAP,326.00,-172.73,65.0,M-SC,2.67,83514.0,-18696.0,112084.0,3.05,-18.29,134.21,91.37,208.0,-0.17,0.61,35.57,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.27,84403.0,-29146.0,69194.0,-0.83,-25.67,81.98,35.27,268.0,-0.42,0.62,106.58,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,12.74,89238.0,-11725.0,106256.0,-0.88,-11.61,119.07,93.63,148.0,-0.11,0.65,32.92,SR,ATH,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3900.00,64.0,M-SC,5.82,124643.0,-22725.0,74537.0,0.86,-15.42,59.80,35.16,236.0,-0.30,0.91,27.31,XY24,NTT,AUTO
60,SAMMAANCAP,326.0,-172.73,65.0,M-SC,2.67,83514.0,-18696.0,112084.0,3.05,-18.29,134.21,91.37,208.0,-0.17,0.61,35.57,XY25,NTT,FINANCE
63,SHALBY,327.0,1099.36,68.0,M-SC,3.64,163077.0,-18066.0,63535.0,3.28,-9.97,38.96,25.10,235.0,-0.28,1.19,30.69,XY24,NTT,HEALTHCARE
21,COFFEEDAY,80.0,-43.66,51.0,L-SC,42.27,84403.0,-29146.0,69194.0,-0.83,-25.67,81.98,35.27,268.0,-0.42,0.62,106.58,XR,NTT,HOTELS
74,TITAGARH,1548.0,0.96,68.0,H-SC,3.26,165638.0,-26377.0,105263.0,0.46,-13.74,63.55,41.08,158.0,-0.25,1.21,40.36,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.11
1,25,43.95
2,50,74.48


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.57
LC    32.16
MC    23.30
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.28
X40      15.44
X40N     11.97
XR       11.81
AR        8.75
XY25      8.26
OX40N     8.23
X200      1.82
MH        1.72
X5K       1.45
SR        1.30
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.33
H-LC    25.45
H-MC    20.32
M-SC    13.65
M-LC     5.65
M-MC     2.67
L-SC     1.59
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.70,-12.33,73.58
FMCG,11.91,-3.96,40.34
FINANCE,9.75,-15.58,60.85
MISC,7.46,-12.70,75.39
BANKS,6.33,-15.59,76.40
PAINTS,5.88,-15.08,32.23
ELECTRICAL,5.77,-4.45,62.01
AC,3.71,4.74,11.22
INSURANCE,3.44,-6.88,43.54


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2843282.0,22
AR,1300319.0,9
XR,1297623.0,14
X40,739399.0,10
X40N,568416.0,9
OX40N,523506.0,10
XY25,333474.0,6
SR,243759.0,2
X5K,119784.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3405550.0,29
M-SC,1379314.0,17
H-LC,1213336.0,14
H-MC,1092384.0,15
M-LC,379364.0,4
M-MC,312777.0,2
L-SC,245014.0,3
L-MC,60681.0,1
L-LC,38781.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1026181.0      6
M-SC       XY24       796396.0      7
H-SC       AR         787439.0      5
           XR         749712.0      7
H-MC       XY24       489548.0      4
H-LC       X40        455923.0      5
           AR         352490.0      2
M-MC       XY24       312777.0      2
H-SC       X40N       284973.0      4
           OX40N      250439.0      4
           SR         243759.0      2
H-MC       X40        212905.0      4
H-LC       X40N       210540.0      3
L-SC       XR         162060.0      2
M-SC       AR         160390.0      2
M-LC       XY24       158474.0      2
M-SC       OX40N      124991.0      4
H-MC       XY25       124900.0      2
M-LC       X5K        119784.0      1
M-SC       XR         114882.0      2
           XY25       112084.0      1
H-MC       XR         109182.0      1
M-LC       XR         101106.0      1
L-SC       OX40N       82954.0      1
H-LC       X200        76768.0      1
H-MC       X40N        72903.0      2
M-SC       X40         70571.0      1
H-MC       OX40N       65122.0      1
H-SC       MH          63047.0      1
L-MC       XR          60681.0      1
H-LC       XY24        59906.0      1
           XY25        57709.0      2
L-LC       XY25        38781.0      1
H-MC       MH          17824.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
